### En este notebook se entrenan diferentes modelos predictivos usando la puntuación OASIS para predecir la mortalidad hospitalaria. Luego se evalua el rendimiento de los modelos con ACC, AUC-ROC, AUC-PR, confusion matrix....

Obs. Ejecute primero '03calculateOASIS.ipynb' para obtener 'result_24_horas_final.csv'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pylab
import sqlite3
import seaborn as sns
import os
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

matplotlib.style.use('ggplot')

In [2]:
patients=pd.read_csv('/data/mimic/PATIENTS.csv')
print(patients.shape)
print('unique SUBJECT_ID:', patients.SUBJECT_ID.nunique())
patients.head()

(46520, 8)
unique SUBJECT_ID: 46520


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [3]:
admissions=pd.read_csv('/data/mimic/ADMISSIONS.csv')
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
admissions.head()

(58976, 19)
unique SUBJECT_ID: 46520
unique HADM_ID   : 58976


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [4]:
admissions = admissions[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DEATHTIME']]
admissions.head(5)

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN


In [5]:
admissions=pd.merge(patients, admissions, on=['SUBJECT_ID'])
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
admissions.head()

(58976, 12)
unique SUBJECT_ID: 46520
unique HADM_ID   : 58976


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,NaN
1,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaN
2,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaN
3,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1,124271,2188-11-12 09:22:00,2188-11-22 12:00:00,2188-11-22 12:00:00
4,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0,117937,2110-07-27 06:46:00,2110-07-29 15:23:00,NaN


In [6]:
admissions = admissions.drop(['ROW_ID','GENDER','DOD_HOSP','DOD_SSN','EXPIRE_FLAG'],axis=1)
admissions.head(5)

,SUBJECT_ID,DOB,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME
0,249,2075-03-13 00:00:00,NaN,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,NaN
1,249,2075-03-13 00:00:00,NaN,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaN
2,249,2075-03-13 00:00:00,NaN,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaN
3,250,2164-12-27 00:00:00,2188-11-22 00:00:00,124271,2188-11-12 09:22:00,2188-11-22 12:00:00,2188-11-22 12:00:00
4,251,2090-03-15 00:00:00,NaN,117937,2110-07-27 06:46:00,2110-07-29 15:23:00,NaN


In [7]:
icustays=pd.read_csv('/data/mimic/ICUSTAYS.csv')
print(icustays.shape)
print('unique SUBJECT_ID:', icustays.SUBJECT_ID.nunique())
print('unique HADM_ID   :', icustays.HADM_ID.nunique())
print('unique ICUSTAY_ID:', icustays.ICUSTAY_ID.nunique())
icustays.head()

(61532, 12)
unique SUBJECT_ID: 46476
unique HADM_ID   : 57786
unique ICUSTAY_ID: 61532


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [8]:
icustays=icustays.drop(['ROW_ID','DBSOURCE', 'FIRST_CAREUNIT','LAST_CAREUNIT','FIRST_WARDID', 'LAST_WARDID','LOS'], axis=1)

In [9]:
admissions=pd.merge(admissions, icustays, on=['SUBJECT_ID', 'HADM_ID'])
print(admissions.shape)
print('unique SUBJECT_ID:', admissions.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions.HADM_ID.nunique())
print('unique ICUSTAY_ID:', admissions.ICUSTAY_ID.nunique())
admissions.head()

(61532, 10)
unique SUBJECT_ID: 46476
unique HADM_ID   : 57786
unique ICUSTAY_ID: 61532


,SUBJECT_ID,DOB,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ICUSTAY_ID,INTIME,OUTTIME
0,249,2075-03-13 00:00:00,NaN,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,NaN,215044,2149-12-18 20:06:02,2149-12-24 13:31:45
1,249,2075-03-13 00:00:00,NaN,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaN,269035,2155-02-03 20:17:29,2155-02-05 18:34:02
2,249,2075-03-13 00:00:00,NaN,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaN,263055,2155-02-07 18:51:16,2155-02-11 16:00:39
3,249,2075-03-13 00:00:00,NaN,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaN,282599,2156-05-01 18:10:12,2156-05-03 18:43:45
4,249,2075-03-13 00:00:00,NaN,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaN,263882,2156-05-10 17:47:35,2156-05-11 19:16:03


In [10]:
admissions['DOD'] = pd.to_datetime(admissions['DOD'])
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'])
admissions['DEATHTIME'] = pd.to_datetime(admissions['DEATHTIME'])

In [11]:
admissions['MORTALITY_INHOSPITAL'] = admissions['DOD'].notnull() & ((admissions['ADMITTIME'] <= admissions['DOD']) & (admissions['DISCHTIME'] >= admissions['DOD']))
admissions['MORTALITY_INHOSPITAL'] = admissions['MORTALITY_INHOSPITAL'] | (admissions['DEATHTIME'].notnull() & ((admissions['ADMITTIME'] <= admissions['DEATHTIME']) & (admissions['DISCHTIME'] >= admissions['DEATHTIME'])))
admissions['MORTALITY_INHOSPITAL'] = admissions['MORTALITY_INHOSPITAL'].astype(int)

In [12]:
admissions['MORTALITY_INHOSPITAL'].value_counts()  #OBSERVACION: tenemos 61532 filas con icustays distintas

0    54948
1     6584
Name: MORTALITY_INHOSPITAL, dtype: int64

In [13]:
admissions['INTIME'] = pd.to_datetime(admissions['INTIME'])
admissions['OUTTIME'] = pd.to_datetime(admissions['OUTTIME'])

In [14]:
admissions['MORTALITY_INUNIT'] = admissions['DOD'].notnull() & ((admissions['INTIME'] <= admissions['DOD']) & (admissions['OUTTIME'] >= admissions['DOD']))
admissions['MORTALITY_INUNIT'] = admissions['MORTALITY_INUNIT'] | (admissions['DEATHTIME'].notnull() & ((admissions['INTIME'] <= admissions['DEATHTIME']) & (admissions['OUTTIME'] >= admissions['DEATHTIME'])))
admissions['MORTALITY_INUNIT'] = admissions['MORTALITY_INUNIT'].astype(int)

In [15]:
admissions['MORTALITY_INUNIT'].value_counts()

0    56845
1     4687
Name: MORTALITY_INUNIT, dtype: int64

In [16]:
admissions

,SUBJECT_ID,DOB,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ICUSTAY_ID,INTIME,OUTTIME,MORTALITY_INHOSPITAL,MORTALITY_INUNIT
0,249,2075-03-13 00:00:00,NaT,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,215044,2149-12-18 20:06:02,2149-12-24 13:31:45,0,0
1,249,2075-03-13 00:00:00,NaT,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaT,269035,2155-02-03 20:17:29,2155-02-05 18:34:02,0,0
2,249,2075-03-13 00:00:00,NaT,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaT,263055,2155-02-07 18:51:16,2155-02-11 16:00:39,0,0
3,249,2075-03-13 00:00:00,NaT,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaT,282599,2156-05-01 18:10:12,2156-05-03 18:43:45,0,0
4,249,2075-03-13 00:00:00,NaT,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaT,263882,2156-05-10 17:47:35,2156-05-11 19:16:03,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
61527,44089,2026-05-25 00:00:00,NaT,165748,2111-09-30 12:04:00,2111-10-03 16:04:00,NaT,292151,2111-09-30 18:53:12,2111-10-01 20:03:50,0,0
61528,44115,2124-07-27 00:00:00,NaT,163623,2161-07-15 12:00:00,2161-07-19 12:30:00,NaT,264947,2161-07-15 17:16:25,2161-07-16 13:21:38,0,0
61529,44123,2049-11-26 00:00:00,2135-01-12,116395,2135-01-06 07:15:00,2135-01-12 02:50:00,2135-01-12 02:50:00,264065,2135-01-06 09:54:24,2135-01-12 04:50:52,1,1
61530,44126,2076-07-25 00:00:00,NaT,183530,2129-01-03 07:15:00,2129-01-11 12:29:00,NaT,241939,2129-01-05 16:51:31,2129-01-07 18:03:39,0,0


In [17]:
result_24_horas_final=pd.read_csv('result_24_horas_final.csv')

In [18]:
result_24_horas_final.isnull().sum()

Unnamed: 0.1          0
Unnamed: 0            0
SUBJECT_ID            0
HADM_ID               0
ADMITTIME             0
ADMISSION_TYPE        0
ICUSTAY_ID            0
INTIME                0
LOS                  10
AGE                   0
PRELOS                0
OASIS             13655
OASIS_NONAN           0
dtype: int64

In [19]:
result_24_horas_final = result_24_horas_final[result_24_horas_final['LOS'].notnull()] 
result_24_horas_final.isnull().sum()

Unnamed: 0.1          0
Unnamed: 0            0
SUBJECT_ID            0
HADM_ID               0
ADMITTIME             0
ADMISSION_TYPE        0
ICUSTAY_ID            0
INTIME                0
LOS                   0
AGE                   0
PRELOS                0
OASIS             13645
OASIS_NONAN           0
dtype: int64

In [20]:
result_24_horas_final = result_24_horas_final[['SUBJECT_ID','HADM_ID','ICUSTAY_ID','OASIS_NONAN']]
result_24_horas_final = result_24_horas_final.reset_index(drop=True)
result_24_horas_final

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,OASIS_NONAN
0,20150,181574,201858,0.0
1,1472,161232,244372,35.0
2,81875,143469,238281,24.0
3,2575,147416,275754,6.0
4,56209,117730,228215,42.0
...,...,...,...,...
61517,27539,185042,279501,30.0
61518,80156,171470,263893,34.0
61519,24938,154322,236223,35.0
61520,21514,108728,296621,27.0


## Predecimos in-hospital-mortality

### Cohort Selection:

In [21]:
#we compute the age at the time the patient is admitted to the ICU from date of birth dob, then we drop dob.
admissions['DOB'] = pd.to_datetime(admissions['DOB']).dt.date
admissions['INTIME_TMP'] = pd.to_datetime(admissions['INTIME']).dt.date
admissions['AGE'] = admissions.apply(lambda e: (e['INTIME_TMP'] - e['DOB']).days/365, axis=1)
admissions=admissions.drop(['DOB','INTIME_TMP'], axis=1)

In [22]:
#Excluir admissions < 18 years old
admissions_filtradas = admissions[~(admissions['AGE'] < 18)]
print(admissions_filtradas.shape)
print('unique SUBJECT_ID:', admissions_filtradas.SUBJECT_ID.nunique())
print('unique HADM_ID   :', admissions_filtradas.HADM_ID.nunique())
print('unique ICUSTAY_ID:', admissions_filtradas.ICUSTAY_ID.nunique())
admissions_filtradas.head()

(53332, 12)
unique SUBJECT_ID: 38512
unique HADM_ID   : 49695
unique ICUSTAY_ID: 53332


,SUBJECT_ID,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ICUSTAY_ID,INTIME,OUTTIME,MORTALITY_INHOSPITAL,MORTALITY_INUNIT,AGE
0,249,NaT,116935,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,215044,2149-12-18 20:06:02,2149-12-24 13:31:45,0,0,74.816438
1,249,NaT,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaT,269035,2155-02-03 20:17:29,2155-02-05 18:34:02,0,0,79.947945
2,249,NaT,149546,2155-02-03 20:16:00,2155-02-14 11:15:00,NaT,263055,2155-02-07 18:51:16,2155-02-11 16:00:39,0,0,79.958904
3,249,NaT,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaT,282599,2156-05-01 18:10:12,2156-05-03 18:43:45,0,0,81.189041
4,249,NaT,158975,2156-04-27 15:33:00,2156-05-14 15:30:00,NaT,263882,2156-05-10 17:47:35,2156-05-11 19:16:03,0,0,81.213699


In [23]:
admissions_filtradas.loc[admissions_filtradas['SUBJECT_ID'] == 59618]
#Ejemplo de un SUBJECT_ID con dos ICUSTAY_ID pero son de HADM_ID distintas.

,SUBJECT_ID,DOD,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ICUSTAY_ID,INTIME,OUTTIME,MORTALITY_INHOSPITAL,MORTALITY_INUNIT,AGE
34933,59618,2102-02-11,101145,2102-01-20 20:38:00,2102-01-25 15:00:00,NaT,226547,2102-01-20 20:39:57,2102-01-21 20:33:19,0,0,74.430137
34934,59618,2102-02-11,157029,2102-02-09 18:14:00,2102-02-11 01:00:00,2102-02-11 01:00:00,275273,2102-02-09 18:15:37,2102-02-11 01:42:05,1,1,74.484932


### Predecir  in-hospital-mortality  con oasis calculada en 24  horas  (considerando los oasis_variable nan como 0.0 (usar np.nansum), i.e. en rango 'normal') + dataset original, no balanceado + mismo cohorte que benchmark

In [24]:
result_24_horas_final=pd.merge(result_24_horas_final, admissions_filtradas, on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'])

In [25]:
print(result_24_horas_final.shape)
print('unique SUBJECT_ID:', result_24_horas_final.SUBJECT_ID.nunique())
print('unique HADM_ID   :', result_24_horas_final.HADM_ID.nunique())
print('unique ICUSTAY_ID   :', result_24_horas_final.ICUSTAY_ID.nunique())
result_24_horas_final.head()

(53329, 13)
unique SUBJECT_ID: 38510
unique HADM_ID   : 49692
unique ICUSTAY_ID   : 53329


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,OASIS_NONAN,DOD,ADMITTIME,DISCHTIME,DEATHTIME,INTIME,OUTTIME,MORTALITY_INHOSPITAL,MORTALITY_INUNIT,AGE
0,1472,161232,244372,35.0,2107-10-03,2107-09-26 15:46:00,2107-10-03 10:15:00,2107-10-03 10:15:00,2107-10-01 21:58:11,2107-10-02 17:52:54,1,0,80.646575
1,81875,143469,238281,24.0,NaT,2193-09-03 19:30:00,2193-09-07 16:50:00,NaT,2193-09-03 19:30:45,2193-09-04 19:43:22,0,0,77.698630
2,56209,117730,228215,42.0,NaT,2179-09-04 08:18:00,2179-09-10 16:38:00,NaT,2179-09-04 08:19:18,2179-09-07 20:48:47,0,0,18.254795
3,5056,156002,210815,29.0,2171-05-21,2171-05-01 17:29:00,2171-05-21 12:00:00,2171-05-21 12:00:00,2171-05-16 09:39:01,2171-05-21 14:25:12,1,1,39.641096
4,19228,138019,240880,19.0,NaT,2193-07-23 18:49:00,2193-08-01 17:19:00,NaT,2193-07-26 17:57:04,2193-07-29 14:48:59,0,0,51.241096


In [26]:
result_24_horas_nonnan=result_24_horas_final[['MORTALITY_INHOSPITAL','OASIS_NONAN','ICUSTAY_ID']]
result_24_horas_nonnan.head()

,MORTALITY_INHOSPITAL,OASIS_NONAN,ICUSTAY_ID
0,1,35.0,244372
1,0,24.0,238281
2,0,42.0,228215
3,1,29.0,210815
4,0,19.0,240880


In [27]:
result_24_horas_nonnan['MORTALITY_INHOSPITAL'].value_counts()

0    46814
1     6515
Name: MORTALITY_INHOSPITAL, dtype: int64

In [28]:
result_24_horas_nonnan=result_24_horas_nonnan.dropna()
result_24_horas_nonnan.shape

(53329, 3)

Normalmente basta con train_test_split para tener conjunto de entrenamiento (Train set) y conjunto de prueba (Test set). Pero como queremos comparar con otros modelos, recomienda mantener un mismo Test set. Por lo tanto, usamos ficheros llamados 'listfilexx' para poder rastrear el Test/Train set. (obs. estas listfiles se obtienen ejecutando create_in_hospital_mortality_icustayidListFile.py)

In [29]:
#from sklearn.model_selection import train_test_split
#from sklearn import preprocessing
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [30]:
cohorte_test_benchmark = pd.read_csv('/data/codi/OASIS/mortality/test/listfileMortality.csv')
cohorte_test_benchmark.rename(columns={'icustayid': 'ICUSTAY_ID'}, inplace=True)
print(cohorte_test_benchmark.shape)
cohorte_test_benchmark.head()

(3236, 3)


,stay,y_true,ICUSTAY_ID
0,10011_episode1_timeseries.csv,1,232110
1,10026_episode1_timeseries.csv,0,277021
2,10030_episode1_timeseries.csv,0,262782
3,10042_episode1_timeseries.csv,0,258147
4,10094_episode1_timeseries.csv,0,243600


In [31]:
test_set=pd.merge(cohorte_test_benchmark, result_24_horas_nonnan, how='left', on=['ICUSTAY_ID'])
print(test_set.shape)
test_set.head()

(3236, 5)


,stay,y_true,ICUSTAY_ID,MORTALITY_INHOSPITAL,OASIS_NONAN
0,10011_episode1_timeseries.csv,1,232110,1,27.0
1,10026_episode1_timeseries.csv,0,277021,0,34.0
2,10030_episode1_timeseries.csv,0,262782,0,31.0
3,10042_episode1_timeseries.csv,0,258147,0,33.0
4,10094_episode1_timeseries.csv,0,243600,0,41.0


In [32]:
test_set.isnull().sum()

stay                    0
y_true                  0
ICUSTAY_ID              0
MORTALITY_INHOSPITAL    0
OASIS_NONAN             0
dtype: int64

In [33]:
test_set[test_set['y_true'] == test_set['MORTALITY_INHOSPITAL']].shape

(3236, 5)

In [34]:
#Solo hace falta ejecutar esta linea una vez, que genera listfile para modelos con times_series
#test_set['ICUSTAY_ID'].to_csv('/home/usuario/TFG/code/final_versiones_OASIS_mismoCohorteBenchmark/mortality/test/listfile.csv', index=False,header='ICUSTAY_ID')

In [35]:
X_test=pd.DataFrame(test_set['OASIS_NONAN'])
y_test=pd.DataFrame(test_set['MORTALITY_INHOSPITAL'])

In [36]:
cohorte_train_benchmark = pd.read_csv('/data/codi/OASIS/mortality/train/listfileMortality.csv')
cohorte_train_benchmark.rename(columns={'icustayid': 'ICUSTAY_ID'}, inplace=True)
print(cohorte_train_benchmark.shape)
cohorte_train_benchmark.head()

(17903, 3)


,stay,y_true,ICUSTAY_ID
0,84984_episode1_timeseries.csv,1,219326
1,57299_episode1_timeseries.csv,0,273321
2,72032_episode1_timeseries.csv,0,221917
3,22175_episode1_timeseries.csv,0,269490
4,13033_episode14_timeseries.csv,0,225465


In [37]:
train_set=pd.merge(cohorte_train_benchmark, result_24_horas_nonnan, how='left', on=['ICUSTAY_ID'])
print(train_set.shape)
train_set.head()

(17903, 5)


,stay,y_true,ICUSTAY_ID,MORTALITY_INHOSPITAL,OASIS_NONAN
0,84984_episode1_timeseries.csv,1,219326,1,40.0
1,57299_episode1_timeseries.csv,0,273321,0,26.0
2,72032_episode1_timeseries.csv,0,221917,0,31.0
3,22175_episode1_timeseries.csv,0,269490,0,33.0
4,13033_episode14_timeseries.csv,0,225465,0,27.0


In [38]:
train_set.isnull().sum()

stay                    0
y_true                  0
ICUSTAY_ID              0
MORTALITY_INHOSPITAL    0
OASIS_NONAN             0
dtype: int64

In [39]:
train_set[train_set['y_true'] == train_set['MORTALITY_INHOSPITAL']].shape

(17903, 5)

In [40]:
#Solo hace falta ejecutar esta linea una vez, que genera listfile para modelos con times_series
#train_set['ICUSTAY_ID'].to_csv('/home/usuario/TFG/code/final_versiones_OASIS_mismoCohorteBenchmark/mortality/train/listfile.csv', index=False,header='ICUSTAY_ID')

In [41]:
X_train=pd.DataFrame(train_set['OASIS_NONAN'])
y_train=pd.DataFrame(train_set['MORTALITY_INHOSPITAL'])

In [42]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [43]:
X_train

array([[ 0.59792936],
       [-0.79294034],
       [-0.29620116],
       ...,
       [ 0.29988585],
       [-1.28967952],
       [-0.395549  ]])

In [44]:
X_test

array([[-0.6935925 ],
       [ 0.00184234],
       [-0.29620116],
       ...,
       [ 0.20053801],
       [ 0.69727719],
       [ 1.59140771]])

## Visualization

In [47]:
from sklearn.decomposition import PCA
# Plotting the Clusters using matplotlib
n=1
pca = PCA(n)
print("Train shape:", X_train.shape)
pca_results = pca.fit_transform(X_train)
print("Nº components:", pca_results.shape[1])
print(pca.explained_variance_ratio_)
print("Cumsum:", np.sum(pca.explained_variance_ratio_))

labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

Train shape: (17903, 1)
Nº components: 1
[1.]
Cumsum: 1.0


In [ ]:
import plotly.express as px

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
components = tsne.fit_transform(X_train)
print(components.shape)

In [ ]:
df = pd.DataFrame()
df["y"] = y_train
df['tsne-one'] = components[:,0]
df['tsne-two'] = components[:,1] 

sns.scatterplot(x="tsne-one", y="tsne-two", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 2),
                s=10,,
                alpha=0.3,
                data=df).set(title="OASIS data T-SNE projection") 

## Models

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

# Silhouette
silhouette_coefficients = []
kmeans_kwargs= {
    "init":"random",
    "n_init":10,
    "max_iter":300,
    "random_state":42
}
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(X_train)
    score = silhouette_score(X_train, kmeans.labels_)
    silhouette_coefficients.append(score)

# Plotting graph to choose the best number of clusters
# with the most Silhouette Coefficient score
print(silhouette_coefficients)
plt.style.use("fivethirtyeight")
plt.plot(range(2, 10), silhouette_coefficients)
plt.xticks(range(2, 10))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [49]:
from sklearn.cluster import KMeans
# KMeans K=2
n_clusters = 2
kmeans = KMeans(
    init="random",
    n_clusters=n_clusters,
    n_init=10,
    max_iter=300,
    random_state=42
)
# Fit to the training data
kmeans.fit(X_train)
train_df = pd.DataFrame(X_train)
# Generate out clusters
train_cluster = kmeans.predict(X_train)
n_columns = X_train.shape[1]

# Add predicted cluster and y regression label to our training DataFrame
train_df.insert(n_columns,'cluster', train_cluster)
train_df.insert(n_columns+1,'y', y_train) 
train_clusters_df = []
for i in range(n_clusters):
    train_clusters_df.append(train_df[train_df['cluster']==i])

# Random Forest
logreg=RandomForestClassifier(n_estimators=10, random_state = 42)
k_logregs = []

In [50]:
# One model for each cluster
print("-------- Logistic Regression fitting for each cluster --------")
for cluster_df in train_clusters_df:
    cluster_X = cluster_df.iloc[:,:n_columns] # X values
    cluster_Y = cluster_df.iloc[:,n_columns+1] # Y value
    logreg.fit(cluster_X, cluster_Y)
    k_logregs.append(logreg)
    print("Training cluster shape: ", cluster_df.shape)

-------- Logistic Regression fitting for each cluster --------
Training cluster shape:  (10302, 3)
Training cluster shape:  (7601, 3)


In [51]:
# Test sample es asignado al cluster correspondiente mediante Distancia euclidiana y se aplica el modelo correspondiente
print("Assigning each test sample to the closest cluster centroid...")
test_df = pd.DataFrame(X_test)
test_clusters_df = test_df.copy()
test_clusters_df["cluster"] = None
test_clusters_df.insert(n_columns+1,'y', y_test) 
i=0
for row in test_df.itertuples(index=False):
    min_distance = float('inf')
    closest_cluster = None
    for k in range(kmeans.cluster_centers_.shape[0]):
        # Check if the assigned cluster has more than 100 samples
        # if train_clusters_df[k].shape[0] > 100: # Probar sin limite
        distance = np.linalg.norm(kmeans.cluster_centers_[k]-row)
        if distance < min_distance:
            min_distance = distance
            closest_cluster = k
    # Assign cluster to test sample
    test_clusters_df.iloc[i, n_columns] = closest_cluster
    i += 1

test_clusters = []
for i in range(n_clusters):
    test_cluster_i = test_clusters_df[test_clusters_df['cluster']==i]
    if np.size(test_cluster_i) != 0:
        test_clusters.append(test_cluster_i)
        print("Test cluster shape: ", test_cluster_i.shape)

Assigning each test sample to the closest cluster centroid...
Test cluster shape:  (1901, 3)
Test cluster shape:  (1335, 3)


In [52]:
def print_metrics_binary(y_true, predictions, verbose=1):
    predictions = np.array(predictions)
    if len(predictions.shape) == 1:
        predictions = np.stack([1 - predictions, predictions]).transpose((1, 0))

    cf = metrics.confusion_matrix(y_true, predictions.argmax(axis=1))
    if verbose:
        print("confusion matrix:")
        print(cf)
    cf = cf.astype(np.float32)

    acc = (cf[0][0] + cf[1][1]) / np.sum(cf)
    prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
    prec1 = cf[1][1] / (cf[1][1] + cf[0][1])
    rec0 = cf[0][0] / (cf[0][0] + cf[0][1])
    rec1 = cf[1][1] / (cf[1][1] + cf[1][0])
    auroc = metrics.roc_auc_score(y_true, predictions[:, 1])

    (precisions, recalls, thresholds) = metrics.precision_recall_curve(y_true, predictions[:, 1])
    auprc = metrics.auc(recalls, precisions)
    minpse = np.max([min(x, y) for (x, y) in zip(precisions, recalls)])

    if verbose:
        print("accuracy = {}".format(acc))
        print("precision class 0 = {}".format(prec0))
        print("precision class 1 = {}".format(prec1))
        print("recall class 0 = {}".format(rec0))
        print("recall class 1 = {}".format(rec1))
        print("AUC of ROC = {}".format(auroc))
        print("AUC of PRC = {}".format(auprc))
        print("min(+P, Se) = {}".format(minpse))

    return {"acc": acc,
            "prec0": prec0,
            "prec1": prec1,
            "rec0": rec0,
            "rec1": rec1,
            "auroc": auroc,
            "auprc": auprc,
            "minpse": minpse}

In [53]:
print("-------- Train metrics ---------")
i = 0
# For each cluster, predict probabilities of class labels
for cluster_df in train_clusters_df:
    cluster_X = cluster_df.iloc[:,:n_columns] # X values
    cluster_Y = cluster_df.iloc[:,n_columns+1] # Y value
    classes_prob = k_logregs[i].predict_proba(cluster_X)
    if i == 0:
        train_X_probs = np.array(classes_prob)
        y_train = np.array(cluster_Y)
    else: 
        train_X_probs = np.concatenate((train_X_probs, classes_prob))
        y_train = np.concatenate((y_train, cluster_Y))
    i += 1
# Print and save the results
ret = print_metrics_binary(y_train, train_X_probs)        


# Test metrics
print("-------- Test metrics ---------")
i = 0
# For each cluster, predict probabilities of class labels
for cluster_df in test_clusters:
    cluster_X = cluster_df.iloc[:,:n_columns] # X values
    cluster_Y = cluster_df.iloc[:,n_columns+1] # Y value
    classes_prob = k_logregs[i].predict_proba(cluster_X)[:,1]
    if i == 0:
        test_X_probs = np.array(classes_prob)
        y_test = np.array(cluster_Y)
    else: 
        test_X_probs = np.concatenate((test_X_probs, classes_prob))
        y_test = np.concatenate((y_test, cluster_Y))
    i += 1
# Print and save the results
ret = print_metrics_binary(y_test, test_X_probs)

-------- Train metrics ---------
confusion matrix:
[[15456    24]
 [ 2380    43]]
accuracy = 0.865720808506012
precision class 0 = 0.866562008857727
precision class 1 = 0.641791045665741
recall class 0 = 0.9984496235847473
recall class 1 = 0.017746595665812492
AUC of ROC = 0.6539229189261823
AUC of PRC = 0.27435324914035014
min(+P, Se) = 0.30568356374807987
-------- Test metrics ---------
confusion matrix:
[[2859    3]
 [ 367    7]]
accuracy = 0.8856613039970398
precision class 0 = 0.886236846446991
precision class 1 = 0.699999988079071
recall class 0 = 0.99895179271698
recall class 1 = 0.01871657744050026
AUC of ROC = 0.6278232752983031
AUC of PRC = 0.23051284216568332
min(+P, Se) = 0.2536231884057971


Random Forest same class weight

In [ ]:
clf=RandomForestClassifier(n_estimators=10, random_state = 42)
clf.fit(X_train,np.ravel(y_train))
y_pred=clf.predict(X_test)
test_ac=np.round(metrics.accuracy_score(y_test, y_pred)*100,4)
train_ac=np.round(clf.score(X_train, y_train)*100,4)
print("Accuracy test:",test_ac)
print("Accuracy train:",train_ac)

In [ ]:
y_pred_probas=clf.predict_proba(X_test)
y_pred_probas #probability of being [class 0, class 1]

In [ ]:
y_pred_probas[:, 1] #probability of being class 1(dead)
#Nota: usamos probabilities para calculo de auc_roc. Porque en el benchmark usa probabilities y además
#el resultado de auc_roc varia si usa probilidad o valores enteros de 0/1.

In [ ]:
np.array(np.ravel(y_test))

In [ ]:
auroc = metrics.roc_auc_score(y_test, y_pred_probas[:, 1])
print("AUC-ROC: ", auroc)
(precisions, recalls, thresholds) = metrics.precision_recall_curve(y_test, y_pred_probas[:, 1])
auprc = metrics.auc(recalls, precisions)
print("AUC-PR: ", auprc)


In [ ]:
confusionMatrix = confusion_matrix(y_test, y_pred)
print(confusionMatrix)


In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)
score = np.round(clf.score(X_test, y_test)*100,4)

In [ ]:
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(score)
plt.title(all_sample_title, size = 15)
plt.show()

In [ ]:
def save_results(pred, y_true, path):
    directory = os.path.dirname(path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(path, 'w') as f:
        f.write("prediction,y_true\n")
        for (x, y) in zip(pred, y_true):
            f.write("{:.6f},{}\n".format(x, y))

In [ ]:
predictions_output_dir = '/home/usuario/TFG/code/final_versiones_OASIS_mismoCohorteBenchmark/mortality'

In [ ]:
save_results(y_pred_probas[:, 1], np.array(np.ravel(y_test)), os.path.join(predictions_output_dir, 'test_predictions', 'RF.csv'))

Random Forest different class weight

In [ ]:
clf=RandomForestClassifier(n_estimators=10, random_state = 42,class_weight='balanced')
clf.fit(X_train,np.ravel(y_train))
y_pred=clf.predict(X_test)
test_ac=np.round(metrics.accuracy_score(y_test, y_pred)*100,4)
train_ac=np.round(clf.score(X_train, y_train)*100,4)
print("Accuracy test:",test_ac)
print("Accuracy train:",train_ac)

In [ ]:
y_pred_probas=clf.predict_proba(X_test)
auroc = metrics.roc_auc_score(y_test, y_pred_probas[:, 1])
print("AUC-ROC: ", auroc)
(precisions, recalls, thresholds) = metrics.precision_recall_curve(y_test, y_pred_probas[:, 1])
auprc = metrics.auc(recalls, precisions)
print("AUC-PR: ", auprc)


In [ ]:
confusionMatrix = confusion_matrix(y_test, y_pred)
print(confusionMatrix)


In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)
score = np.round(clf.score(X_test, y_test)*100,4)

In [ ]:
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(score)
plt.title(all_sample_title, size = 15)
plt.show()

In [ ]:
save_results(y_pred_probas[:, 1], np.array(np.ravel(y_test)), os.path.join(predictions_output_dir, 'test_predictions', 'RF_weighted.csv'))

Logistic Regression same class weight

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, np.ravel(y_train))
y_pred = logreg.predict(X_test)
actr=np.round(100*logreg.score(X_train, y_train),4)
acte=np.round(100*logreg.score(X_test, y_test),4)
print('Accuracy of logistic regression classifier on train set:', actr)
print('Accuracy of logistic regression classifier on test set:', acte)


In [ ]:
y_pred_probas=logreg.predict_proba(X_test)
auroc = metrics.roc_auc_score(y_test, y_pred_probas[:, 1])
print("AUC-ROC: ", auroc)
(precisions, recalls, thresholds) = metrics.precision_recall_curve(y_test, y_pred_probas[:, 1])
auprc = metrics.auc(recalls, precisions)
print("AUC-PR: ", auprc)


In [ ]:
confusionMatrix = confusion_matrix(y_test, y_pred)
print(confusionMatrix)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
save_results(y_pred_probas[:, 1], np.array(np.ravel(y_test)), os.path.join(predictions_output_dir, 'test_predictions', 'LR.csv'))

Logistic Regression different class weight

In [ ]:
logreg = LogisticRegression(class_weight='balanced')
logreg.fit(X_train, np.ravel(y_train))
y_pred = logreg.predict(X_test)
actr=np.round(100*logreg.score(X_train, y_train),4)
acte=np.round(100*logreg.score(X_test, y_test),4)
print('Accuracy of logistic regression classifier on train set:', actr)
print('Accuracy of logistic regression classifier on test set:', acte)

In [ ]:
y_pred_probas=logreg.predict_proba(X_test)
auroc = metrics.roc_auc_score(y_test, y_pred_probas[:, 1])
print("AUC-ROC: ", auroc)
(precisions, recalls, thresholds) = metrics.precision_recall_curve(y_test, y_pred_probas[:, 1])
auprc = metrics.auc(recalls, precisions)
print("AUC-PR: ", auprc)


In [ ]:
confusionMatrix = confusion_matrix(y_test, y_pred)
print(confusionMatrix)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
save_results(y_pred_probas[:, 1], np.array(np.ravel(y_test)), os.path.join(predictions_output_dir, 'test_predictions', 'LR_weighted.csv'))